# 深層学習

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
ニューラルネットワークでマルコフモデルを作成し、深層学習における言語モデル実装の基礎を学ぶ。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow_datasets as tfds
import MeCab
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import vocab, build_vocab_from_iterator
from tqdm import tqdm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## データセット

wiki40b

In [2]:
with tf.device('/cpu:0'):
    ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

In [3]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

多すぎるので減らす

In [4]:
data = data[:100]


---

## 前処理

テキストをNNで扱える形に変換する。

### トークン化

文章をトークンごとに分割する。トークンとは文章を構成する最小の単位で、単語や句読点などが該当する。  
トークン化は分かち書きと似た意味であるが、言語モデルの領域で最小単位をトークンと呼ぶことが一般的なことや、単純にトークン化と呼ぶことが多いことから、本節以降トークン化と呼ぶ。

トークン化には、分かち書き同様形態素解析を用いる。各文章をトークンのリストとして格納する。

In [5]:
data_tokens = []
tagger = MeCab.Tagger('-Owakati')
for sentence in data:
    tokens = tagger.parse(sentence).strip().split(' ') # トークンのリスト
    data_tokens.append(tokens)

data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

### ID化

NNでは文字列を扱えないので、トークン1つ1つにIDを割り当てる。

<br>

torchtextの`vocab`モジュールを使用する。

[torchtext.vocab — Torchtext 0.15.0 documentation](https://pytorch.org/text/stable/vocab.html#)

In [6]:
from torchtext.vocab import vocab, build_vocab_from_iterator

単語とその出現頻度を表した辞書を入力し、`Vocab`オブジェクトを作成する。`min_freq`で指定した出現頻度以下の単語は無視される。

In [7]:
v = vocab({'私': 4, 'りんご': 1, '食べる': 2, '好き': 3}, min_freq=1)
v(['私', 'りんご', '好き'])

[0, 1, 3]

`build_vocab_from_iterator()`でトークンのリスト（のリスト）から`vocab`オブジェクトを作成できる。

In [8]:
v = build_vocab_from_iterator([
    ['私', 'は', 'りんご', 'が', '好き'],
    ['私', 'は', 'バナナ', 'を', '食べる'],
    ['今日', 'は', '晴れ', 'です']
])
v.get_itos() # vocabrary

['は', '私', 'が', 'です', 'りんご', 'を', 'バナナ', '今日', '好き', '晴れ', '食べる']

これを使ってトークン列をID列に変換する。

In [9]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

print('num of vocabrary:', len(v))
data_ids[0][:10] # example

num of vocabrary: 4602


[17, 3681, 3742, 2, 4, 3873, 4415, 47, 19, 70]


---

## 学習データ

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        x, y = [], []
        for ids in data_ids:
            for id1, id2 in zip(ids[:-1], ids[1:]):
                x.append(id1)
                y.append(id2)
        self.x = x
        self.y = y
        self.n_data = len(x)

    def __len__(self):
        return self.n_data

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

batch_size = 64
dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

n_data = len(dataset)
print('num of data:', n_data)

# examples
x, y = next(iter(dataloader))
print(x)
print(y)

num of data: 21128
tensor([  18, 1062,   22, 1061,  243, 2584,  216,   23,  137,   64,   18, 4524,
         933, 4482, 1559,  912, 2110, 3530, 1130, 1377,   14,    0, 3499, 1982,
         639,    7, 1159,  513,  352,   19,    4,   57,  827, 3659,   10, 3084,
         199,   23,    4,    4,    0,   11,  759,    1,   19, 1942, 1609,    1,
           1,   16,    0,    2,   10,  271,   18, 1023,  138, 2630,  511,   13,
        1317, 3191,   19, 1604])
tensor([   3,   12, 2623,   12, 2127,   22,  567,  513, 3125,    8,    3,   10,
           8,    0, 1452,  794,  279,    2,   20,    9,   13,  127,    6,   28,
          90, 3894,   35,  712,  256,    7,    1,  145,    5,   14, 1441,   11,
           9,  883,    1,   39, 3399,    6,    4, 2624,    7,  134,  139, 2290,
          39,    4,   35,  347, 3548,  299,    3,    0,    0,   15,    4,    6,
           2,    3,    7,    3])



----

## モデル構築

マルコフモデルはある状態から次の状態を予測する。言語モデルに当てはめると、ある単語から次の単語を予測する。  
これをNNで実装するので、ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

単語IDはカテゴリ変数なので、入力時はone-hotベクトルに変換する。出力は語彙数分の次元を持つベクトルを出力し、単語の分類問題として扱う。

In [11]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.net = nn.Sequential(
            nn.Linear(vocab_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, vocab_size)
        )

    def forward(self, x):
        x = F.one_hot(x, self.vocab_size).to(torch.float32)
        y = self.net(x)
        return y


---

## 学習

通常の分類モデルと同じように学習する。

In [12]:
criterion = nn.CrossEntropyLoss()
def train(model, optimizer, dataloader, n_epochs):
    model.train()
    for epoch in range(1, n_epochs + 1):
        loss_epoch = 0
        for x, t in tqdm(dataloader, desc=f'{epoch}/{n_epochs}', leave=False):
            optimizer.zero_grad()
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = criterion(y, t)
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        loss = loss_epoch/len(dataloader)
        print(f'{epoch}/{n_epochs} loss: {loss}', flush=True)

In [13]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.617839218266421


2/20 loss: 5.613264075218731


3/20 loss: 5.059253672458614


4/20 loss: 4.548879970596636


5/20 loss: 4.112352208428512


...

16/20 loss: 2.7113829447783733


17/20 loss: 2.702974404093002


18/20 loss: 2.698480599236272


19/20 loss: 2.6884260206424218


20/20 loss: 2.686243463859097



---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。ある単語からの次の単語の予測を繰り返すことで文章を生成する。初めの単語だけはこちらで指定する。  
以下の条件を満たすまで単語の生成を続ける。
- 単語数が指定した限界に達する
- 読点が出力される

In [15]:
end_token_id = v.get_stoi()['。'] # 読点のID

def generate_sentence(model, start_word, max_len=30):
    model.eval()
    token_id = v.get_stoi()[start_word]
    token_ids = [token_id]

    # 終了条件を満たすまで単語を生成
    while len(token_ids) > max_len and token_id != end_token_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor(token_id).unsqueeze(0).to(device)

        # 次の単語の確率分布を予測
        y = model(x)[0]
        y = F.softmax(y, dim=0)

        # 分布に基づいたサンプリングを行う
        token_id = random.choices(range(len(y)), weights=y)[0]
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [16]:
start_words = ['今日', '地球', '科学', '人']
for start_word in start_words:
    print(generate_sentence(model, start_word))

今日
地球
科学
人



---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### BOS

*Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### EOS

*End of Sentence*

文章の終わりを意味するトークン。  
先程の例では読点が出た場合に生成を止めたが、本来読点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。

<br>

では特殊トークンを使って学習させてみよう。使う特殊トークン上の二つ。

In [17]:
data_tokens[0][:10] # example

['「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」', '日本']

全ての文章にBOSとEOSを追加する。  
他のトークンと重複しないような文字列を指定する。

In [18]:
BOS, EOS = '<BOS>', '<EOS>'
for tokens in data_tokens:
    tokens.insert(0, BOS)
    tokens.append(EOS)

data_tokens[0][:10] # example

['<BOS>', '「', '教科', '書', 'に', 'は', '決して', '載ら', 'ない', '」']

残りの学習は同じ

In [19]:
data_ids = []
v = build_vocab_from_iterator(data_tokens)
for tokens in data_tokens:
    data_ids.append(v(tokens))

dataset = TextDataset(data_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [20]:
model = LanguageModel(len(v), 512).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
train(model, optimizer, dataloader, 20)

1/20 loss: 6.617031986841899


2/20 loss: 5.612874150990012


3/20 loss: 5.077422079211938


4/20 loss: 4.557763280982742


5/20 loss: 4.119269445270835


...

16/20 loss: 2.722364569495538


17/20 loss: 2.711752150230065


18/20 loss: 2.7037228617125644


19/20 loss: 2.6991396296524


20/20 loss: 2.6947849710544425


では文章を生成する。初めのトークンはBOSとし、EOSが出力されるまで生成を続ける。

In [22]:
end_token_id = v.get_stoi()[EOS]

def generate_sentence(model, max_len=50):
    model.eval()
    token_id = v.get_stoi()[BOS]
    token_ids = []

    for _ in range(max_len):
        x = torch.tensor(token_id).unsqueeze(0).to(device)
        y = model(x)[0]
        y = F.softmax(y, dim=0)
        token_id = random.choices(range(len(y)), weights=y)[0]
        if token_id == end_token_id:
            break
        token_ids.append(token_id)

    tokens = [v.get_itos()[i] for i in token_ids]
    sentence = ''.join(tokens)
    return sentence

In [23]:
for _ in range(5):
    print(generate_sentence(model))

艦サーモンはVSO型である。同じでは殆どの行方は、父は仕事。13時間は「ニュースでは5回（桃井はるこ、キャラクター・LDH）が発生するとなって挙げられることとして
養成所付近に葬られた4人以外では次々という位置づけられた（1885年大会である。2002年から2015年による演奏にはチトラドゥルガ陥落のサム・サン＝までがみー艦隊司令官
マサイ語に移ったために住み、2000億円→同系統の廃止されていた。また、月曜の高氏のホイサラとしていた。これらの主砲を行なったブラウンシュヴァイク＝ピエール・育成
武蔵野美術教師を周知させてサービス開始。だ。カストゥーリ・ランガッパ・ナーヤカ2世（大舘はPS版と再会しており、トライアウトに配置された。1990年）、全国少年少女戦士たちに就けられた改修
メイン練習時間は優柔不断で表された料が花束嬢の歌手で、2010KZ₃₉のユーロビジョンになったが育成に使うことが、ISO3105、2009年と楽曲の記録した。『さらば』の
